In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import dataframe_image as dfi



### Load Data - Wimbledon 2025 Analysis

In [ ]:
player_strengths_male = pd.read_csv('results/player_strengths_male_Wimbledon_2025.csv')
player_strengths_male.columns = ['player_name','strength']
player_strengths_female = pd.read_csv('results/player_strengths_female_Wimbledon_2025.csv')
player_strengths_female.columns = ['player_name','strength']
predicted_round_reached_dataframe_male = pd.read_csv('results/predicted_round_reached_dataframe_Wimbledon_2025_male.csv')
predicted_round_reached_dataframe_female = pd.read_csv('results/predicted_round_reached_dataframe_Wimbledon_2025_female.csv')

wimbledon_male_results = pd.read_csv('src/main_package/data/2025_men.csv')
wimbledon_male_results = wimbledon_male_results[wimbledon_male_results['Tournament'] == 'Wimbledon'].reset_index(drop=True)
wimbledon_female_results = pd.read_csv('src/main_package/data/2025_women.csv')
wimbledon_female_results = wimbledon_female_results[wimbledon_female_results['Tournament'] == 'Wimbledon'].reset_index(drop=True)


## Calculate Rankings

In [ ]:

male_rank_map = {
    **(wimbledon_male_results.set_index('Winner')['WRank'].to_dict()),
    **(wimbledon_male_results.set_index('Loser')['LRank'].to_dict()),
}


female_rank_map = {
    **(wimbledon_female_results.set_index('Winner')['WRank'].to_dict()),
    **(wimbledon_female_results.set_index('Loser')['LRank'].to_dict()),
}

In [ ]:
# Combine dataframes
merged_male_data = pd.merge(player_strengths_male, predicted_round_reached_dataframe_male, on='player_name',how='right')
merged_male_data['rank'] = merged_male_data['player_name'].map(male_rank_map)
merged_female_data = pd.merge(player_strengths_female, predicted_round_reached_dataframe_female, on='player_name',how='right')
merged_female_data['rank'] = merged_female_data['player_name'].map(female_rank_map)

In [ ]:

def create_ranking_plot(male:bool, num_players:int = 20):
    """
    Create a ranking plot, comparing the model ranking to the player's true rank
    """
    if male:
        plot_rankings = merged_male_data.sort_values(by='strength').reset_index(drop=True).iloc[:num_players].iloc[::-1].dropna().reset_index(drop=True)
    else:
        plot_rankings = merged_female_data.sort_values(by='strength').reset_index(drop=True).iloc[:num_players].iloc[::-1].dropna().reset_index(drop=True)

    fig,ax = plt.subplots(figsize = (8,6))
    fig.patch.set_facecolor((0.9,0.9,0.9))
    scaled_scores = plot_rankings['strength']/plot_rankings['strength'].min()

    plt.barh(plot_rankings['player_name'], (-1)*plot_rankings['strength'],ec='k',color = [(1-c,1-c,1) for c in scaled_scores])

    for index, plot_row in plot_rankings.iterrows():
        # Calculate difference
        difference = int(plot_row['rank'] - (len(plot_rankings) - index))
        difference_string = f'+{difference}' if difference > 0 else str(difference)

        # Choose diverging colormap: green → white → red
        # Normalize with 0 at center
        vmax = 10  # max abs value expected
        norm = mcolors.TwoSlopeNorm(vmin=-vmax, vcenter=0, vmax=vmax)

        # Choose diverging colormap: green → white → red
        cmap = cm.get_cmap('RdYlGn_r')  # reversed: green for negative, red for positive
        bg_colour = cmap(norm(-difference))  # RGBA tuple
        # Determine if text should be black or white (based on brightness)
        r, g, b, _ = bg_colour
        brightness = 0.299 * r + 0.587 * g + 0.114 * b
        font_colour = 'black' if brightness > 0.5 else 'white'


        # Draw text with rounded box
        plt.text(
            -plot_row['strength'] + 0.05,
            index,
            difference_string,
            ha='left',
            va='center',
            fontsize=10,
            fontweight='bold',
            color=font_colour,
            bbox=dict(
                boxstyle='round,pad=0.2',
                facecolor=bg_colour,
                edgecolor='k'
            )
        )

    ax.spines[['top', 'right']].set_visible(False)
    
    suffix = 'male' if male else 'female'
    plt.title(f'Top Model Grass-Court Players ({suffix.title()})', fontsize=14, fontweight='bold')
    plt.xlabel('Model Player Score')
    fig.set_size_inches(6,8)
    fig.tight_layout()
    
    plt.savefig(f'ranking_plot_{suffix}.png',bbox_inches='tight',dpi=300)


In [ ]:
create_ranking_plot(True)
create_ranking_plot(False)

## Compare Performance to Rankings

In [ ]:
male_model_rank_map = merged_male_data.set_index('player_name')['strength'].to_dict()
wimbledon_male_results['winner_model_rank'] = wimbledon_male_results['Winner'].map(male_model_rank_map).fillna(max(male_model_rank_map.values()))
wimbledon_male_results['loser_model_rank'] = wimbledon_male_results['Loser'].map(male_model_rank_map).fillna(max(male_model_rank_map.values()))
wimbledon_male_results['full_data'] =(~pd.isna( wimbledon_male_results['Winner'].map(male_model_rank_map)))&(~pd.isna( wimbledon_male_results['Loser'].map(male_model_rank_map)))

In [ ]:
female_model_rank_map = merged_female_data.set_index('player_name')['strength'].to_dict()
wimbledon_female_results['winner_model_rank'] = wimbledon_female_results['Winner'].map(female_model_rank_map).fillna(max(female_model_rank_map.values()))
wimbledon_female_results['loser_model_rank'] = wimbledon_female_results['Loser'].map(female_model_rank_map).fillna(max(female_model_rank_map.values()))
wimbledon_female_results['full_data'] =(~pd.isna( wimbledon_female_results['Winner'].map(female_model_rank_map)))&(~pd.isna( wimbledon_female_results['Loser'].map(female_model_rank_map)))

In [ ]:
combined_results = pd.concat([wimbledon_male_results, wimbledon_female_results])
# Filter NA ranks
combined_results = combined_results[combined_results['winner_model_rank']!=combined_results['loser_model_rank']]

In [ ]:
(combined_results['winner_model_rank'] < combined_results['loser_model_rank']).sum(),(combined_results['WRank'] < combined_results['LRank']).sum()

### Compare Performance to Bookies

In [ ]:
np.sum(combined_results['AvgW'] < combined_results['AvgL'])

In [ ]:
rank_difference= combined_results['winner_model_rank']-combined_results['loser_model_rank']
combined_results['model_prob']=  1/(1 + np.exp(rank_difference))
combined_results["bookmaker_prob"] = (1 / combined_results["AvgW"]).to_numpy() / (
        (1 / combined_results[["AvgW", "AvgL"]].to_numpy()).sum(axis=1)
    )

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(6,6))
fig.patch.set_facecolor((0.9,0.9,0.9))
combined_results['difference'] = np.abs(combined_results['bookmaker_prob'] - combined_results['model_prob'])

plt.scatter(combined_results[combined_results['full_data']]['bookmaker_prob'],
            combined_results[combined_results['full_data']]['model_prob'],
            color = (1,1,1), ec='k',s = 100, label='Full Data')




plt.scatter(combined_results[~combined_results['full_data']]['bookmaker_prob'],
            combined_results[~combined_results['full_data']]['model_prob'],
            color = (0,0,0),label='No Data', ec='k',s = 100)

plt.legend(fontsize=11, facecolor = (0.9,0.9,0.9),edgecolor='k')
plt.xlabel('Bookmaker Probability')
plt.ylabel('Model Probability')
plt.title('Model vs Bookmaker Winner Probability')
ax.spines[['top', 'right']].set_visible(False)
fig.tight_layout()
plt.savefig('probability_comparison.png',dpi=300,bbox_inches='tight')

In [ ]:
np.polyfit(combined_results[combined_results['full_data']]['bookmaker_prob'],combined_results[combined_results['full_data']]['model_prob'],1)

In [ ]:
np.mean(np.log(combined_results[combined_results['full_data']]['bookmaker_prob'])), np.mean(np.log(combined_results[combined_results['full_data']]['model_prob']))

### Generate Outlier Table

In [ ]:
outlier_data = combined_results[(combined_results['full_data'])&(combined_results['difference'] > 0.25)][['Winner','Loser','WRank','LRank','Round','model_prob','bookmaker_prob', 'difference']].sort_values(by='difference',ascending=False)
outlier_data = outlier_data.reset_index(drop=True)
outlier_data.columns = ['Winner','Loser','Winner\nRank','Loser\nRank','Round','Model Prob','Bookmaker Prob', 'Difference']
outlier_data.index += 1

# Round numeric columns to 2 decimal places
outlier_data[["Model Prob", "Bookmaker Prob", "Difference"]] = outlier_data[["Model Prob", "Bookmaker Prob", "Difference"]].round(2)
outlier_data[['Winner\nRank','Loser\nRank']] = outlier_data[['Winner\nRank','Loser\nRank']].astype(int)
# Style with blue gradient
styled_df = outlier_data.style.background_gradient(
    subset=["Model Prob", "Bookmaker Prob", "Difference"],
    cmap="Blues", axis=0
).format(precision=2)


dfi.export(styled_df, 'outlier_table.png',dpi=300, table_conversion='matplotlib')


### Longitudinal Analysis

In [ ]:
model_correct_results = 0
ranking_correct_results = 0
bookmaker_correct_results = 0
bookmaker_brier_score = 0.0
model_brier_score = 0.0
total_results = 0
tournament_statistics:list[pd.Series] = []
for tournament_year in [2024,2025]:
    for tournament in ['Australian Open','French Open','Wimbledon','US Open']:
    
        # No US Open in 2025 in the dataset as it hasn't happened yet!
        if tournament_year == 2025 and tournament == 'US Open':
            continue
        
        # Load the data for the tournament
        player_strengths_male = pd.read_csv(f'results/player_strengths_male_{tournament}_{tournament_year}.csv')
        player_strengths_male.columns = ['player_name','strength']
        player_strengths_female = pd.read_csv(f'results/player_strengths_female_{tournament}_{tournament_year}.csv')
        player_strengths_female.columns = ['player_name','strength']

        wimbledon_male_results = pd.read_csv(f'src/main_package/data/{tournament_year}_men.csv')
        wimbledon_male_results = wimbledon_male_results[wimbledon_male_results['Tournament'] == tournament].reset_index(drop=True)
        wimbledon_female_results = pd.read_csv(f'src/main_package/data/{tournament_year}_women.csv')
        wimbledon_female_results = wimbledon_female_results[wimbledon_female_results['Tournament'] == tournament].reset_index(drop=True)
        
        # Create the rankings map
        male_rank_map = {
            **(wimbledon_male_results.set_index('Winner')['WRank'].to_dict()),
            **(wimbledon_male_results.set_index('Loser')['LRank'].to_dict()),
        }


        female_rank_map = {
            **(wimbledon_female_results.set_index('Winner')['WRank'].to_dict()),
            **(wimbledon_female_results.set_index('Loser')['LRank'].to_dict()),
        }

        # Combine dataframes
        merged_male_data = pd.merge(player_strengths_male, predicted_round_reached_dataframe_male, on='player_name',how='right')
        merged_male_data['rank'] = merged_male_data['player_name'].map(male_rank_map)
        merged_female_data = pd.merge(player_strengths_female, predicted_round_reached_dataframe_female, on='player_name',how='right')
        merged_female_data['rank'] = merged_female_data['player_name'].map(female_rank_map)

        

        # Get the ranking of each player
        male_model_rank_map = merged_male_data.set_index('player_name')['strength'].to_dict()
        wimbledon_male_results['winner_model_rank'] = wimbledon_male_results['Winner'].map(male_model_rank_map).fillna(max(male_model_rank_map.values()))
        wimbledon_male_results['loser_model_rank'] = wimbledon_male_results['Loser'].map(male_model_rank_map).fillna(max(male_model_rank_map.values()))
        female_model_rank_map = merged_female_data.set_index('player_name')['strength'].to_dict()
        wimbledon_female_results['winner_model_rank'] = wimbledon_female_results['Winner'].map(female_model_rank_map).fillna(max(female_model_rank_map.values()))
        wimbledon_female_results['loser_model_rank'] = wimbledon_female_results['Loser'].map(female_model_rank_map).fillna(max(female_model_rank_map.values()))
        wimbledon_female_results['full_data'] =(~pd.isna( wimbledon_female_results['Winner'].map(female_model_rank_map)))&(~pd.isna( wimbledon_female_results['Loser'].map(female_model_rank_map)))

        # Combine male and female results
        combined_results = pd.concat([wimbledon_male_results, wimbledon_female_results])
        
        # Filter matches where the predictions are the same
        combined_results = combined_results[combined_results['winner_model_rank']!=combined_results['loser_model_rank']]

        # Calculate the results prediction percentages
        model_correct_results += (combined_results['winner_model_rank'] < combined_results['loser_model_rank']).sum()
        ranking_correct_results += (combined_results['WRank'] < combined_results['LRank']).sum()
        bookmaker_correct_results += np.sum(combined_results['AvgW'] < combined_results['AvgL'])
        total_results += len(combined_results)

        # Calculate probabilities
        rank_difference= combined_results['winner_model_rank']-combined_results['loser_model_rank']
        combined_results['model_prob']=  1/(1 + np.exp(rank_difference))
        combined_results["bookmaker_prob"] = (1 / combined_results["AvgW"]).to_numpy() / (
                (1 / combined_results[["AvgW", "AvgL"]].to_numpy()).sum(axis=1)
            )

        # Create a combined dataframe
        tournament_statistics.append(pd.Series(
            {
                'tournament':tournament,
                'year':tournament_year,
                'model_perc' : (combined_results['winner_model_rank'] < combined_results['loser_model_rank']).sum()/len(combined_results),
                'bookmaker_perc' : np.sum(combined_results['AvgW'] < combined_results['AvgL'])/len(combined_results),
                'ranking_perc' : (combined_results['WRank'] < combined_results['LRank']).sum()/len(combined_results),
                'total_results':len(combined_results)
            }
        ))

In [ ]:
# Create a plot for each tournament

tournament_df = pd.DataFrame(tournament_statistics)
fig,ax = plt.subplots(figsize = (8,6))
fig.patch.set_facecolor((0.9,0.9,0.9))
plt.bar(np.arange(len(tournament_df))-0.25, tournament_df['bookmaker_perc']*100, width = 0.25, color=(0.0,0.0,0.6), ec='k', label='Bookmakers')
plt.bar(np.arange(len(tournament_df)), tournament_df['model_perc']*100, width = 0.25, color=(1.0, 0.8, 0.5), ec='k', label = 'Model')
plt.bar(np.arange(len(tournament_df))+0.25, tournament_df['ranking_perc']*100, width = 0.25, color=(0.8,0.8,1.0), ec='k', label='Ranking')

plt.xticks(np.arange(len(tournament_df)),['A24','F24','W24','US24','A25','F25','W25'])
plt.ylim([60,80])
plt.xlabel('Tournament')
plt.ylabel('Correct Results Percentage')
plt.title(f'Result Forecast Success By Tournament', fontsize=14, fontweight='bold')
plt.legend()
fig.set_size_inches(8,6)
plt.savefig('results_by_tournament.png',dpi=300,bbox_inches='tight')

In [ ]:
total_results,bookmaker_correct_results,model_correct_results,ranking_correct_results

In [ ]:
from scipy.stats import binomtest
binomtest(k=model_correct_results, n=total_results, p=ranking_correct_results/total_results, alternative='two-sided')

In [ ]:
binomtest(k=model_correct_results, n=total_results, p=bookmaker_correct_results/total_results, alternative='two-sided')

In [ ]:
tournament_df